In [31]:
import numpy as np
from PIL import Image
import glob

IMG_SHAPE = (250, 250)


def img_to_vector(path_to_img):
    # Load the image
    img = Image.open(path_to_img).convert('L')

    # Convert the image into n²*1 array
    arr = np.array(img)
    flat_array = arr.ravel()

    return flat_array


def vector_to_img(vector, shape):
    array = vector.reshape(shape)
    img = Image.fromarray(array)
    return img


def sum_of_vectors(arr: []):
    sum_vector = np.zeros(len(arr[0]))
    for i in range(0, len(arr[0])):
        for v in arr:
            sum_vector[i] += v[i]

    return sum_vector


def scalar_multiply_vector(scalar, v):
    arr = np.array([])
    for x in v:
        arr = np.append(arr, x * scalar)
    return arr


def negative_vector(v):
    new_v = []
    for x in v:
        new_v.append(-x)
    return np.array(new_v)

In [32]:
all_imgs = []

# Get images from dataset and convert them to vectors
for folder in glob.iglob('../dataset/*'):
    for img in glob.iglob(folder + '/*'):
        all_imgs.append(img_to_vector(img))

# Create an np.array from the vectors
training_set = np.array(np.array(all_imgs))


In [33]:
# Average face using numpy
avg_face = training_set.mean(axis=0)

In [34]:
# Let's create the matrix A by subtracting the average face from each face in the training set
A = []
neg_avg_face = negative_vector(avg_face)
for v in training_set:
    A.append(sum_of_vectors([v, neg_avg_face]))

vector_to_img(A[0], shape=IMG_SHAPE).show()

# Convert A to a matrix
A_m = np.asmatrix(A)

A_t = A_m.transpose()


In [35]:
# Form the covariance matrix
cov_matrix = np.matmul(A_m, A_t)

In [66]:
# Calculate the eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]

In [67]:
# Choose the 10 eigenvectors with the highest eigenvalues as the eigenfaces
eigenfaces = np.array(eigvectors_sort[:7]).transpose()

In [68]:
# Calculate eiganfaces
# Create reduced eigenface space
proj_data = np.dot(training_set.transpose(), eigenfaces)
proj_data = proj_data.transpose()

In [47]:
# calculate eigenface of image in question
test_image: np.ndarray = img_to_vector('../dataset/test/George_W_Bush_0001.jpg')
mean_test_image = sum_of_vectors([test_image, neg_avg_face])
vector_to_img(mean_test_image, shape=IMG_SHAPE).show()

In [52]:
# Project the eigfaces to eigspace
proj_data = np.dot(training_set.transpose(), np.asmatrix(eigenfaces_sorted[:7]).transpose())
proj_data = proj_data.transpose()

#for x in proj_data:
#    vector_to_img(x, shape=IMG_SHAPE).show()

ValueError: maximum supported dimension for an ndarray is 32, found 225